In [70]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version

spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,520 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,289 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main

In [71]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [72]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [86]:
# 2. Create a temporary view of the DataFrame.

spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',')
df.select(year(df["date"]))
df = df.withColumn("year", year(df['date']))
df.createOrReplaceTempView('df')
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|2022|
|5aa00529-0533-46b...|2019-01-30|      2

In [89]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
SELECT ROUND(AVG(price), 2), year
FROM df
WHERE bedrooms == 4
GROUP BY year
"""
spark.sql(query).show()

+--------------------+----+
|round(avg(price), 2)|year|
+--------------------+----+
|           296363.88|2022|
|            300263.7|2019|
|           298353.78|2020|
|           301819.44|2021|
+--------------------+----+



In [85]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

query = """
SELECT ROUND(AVG(price), 2), date_built
FROM df
WHERE bedrooms == 3
AND bathrooms ==3
GROUP BY date_built
"""
spark.sql(query).show()

+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|            288770.3|      2015|
|           295962.27|      2013|
|           290852.27|      2014|
|           293683.19|      2012|
|           290555.07|      2016|
|           292859.62|      2010|
|           291117.47|      2011|
|           292676.79|      2017|
+--------------------+----------+



In [90]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

query = """
SELECT ROUND(AVG(price), 2), date_built
FROM df
WHERE bedrooms == 3
AND bathrooms ==3
AND floors ==2
AND sqft_living >= 2000
GROUP BY date_built
"""
spark.sql(query).show()

+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           297609.97|      2015|
|           303676.79|      2013|
|           298264.72|      2014|
|           307539.97|      2012|
|            293965.1|      2016|
|           285010.22|      2010|
|           276553.81|      2011|
|           280317.58|      2017|
+--------------------+----------+



In [101]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()
query = """
SELECT ROUND(AVG(view), 2), date_built
FROM df
WHERE price >= 350000
GROUP BY date_built
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+----------+
|round(avg(view), 2)|date_built|
+-------------------+----------+
|              31.89|      2015|
|              32.61|      2013|
|              32.28|      2014|
|              33.48|      2012|
|              32.91|      2016|
|              30.64|      2010|
|              32.28|      2011|
|              32.03|      2017|
+-------------------+----------+

--- 0.9351201057434082 seconds ---


In [98]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table df')

DataFrame[]

In [99]:
# 8. Check if the table is cached.
spark.catalog.isCached('df')

True

In [102]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
query = """
SELECT ROUND(AVG(view), 2), date_built
FROM df
WHERE price >= 350000
GROUP BY date_built
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))


+-------------------+----------+
|round(avg(view), 2)|date_built|
+-------------------+----------+
|              31.89|      2015|
|              32.61|      2013|
|              32.28|      2014|
|              33.48|      2012|
|              32.91|      2016|
|              30.64|      2010|
|              32.28|      2011|
|              32.03|      2017|
+-------------------+----------+

--- 0.39283132553100586 seconds ---


In [106]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.parquet('parquet_data', mode='overwrite')
df.write.partitionBy("date_built").mode("overwrite").parquet("data_partitioned")

In [111]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('parquet_data')

In [112]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_data')

In [113]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query = """
SELECT ROUND(AVG(view), 2), date_built
FROM df
WHERE price >= 350000
GROUP BY date_built
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+----------+
|round(avg(view), 2)|date_built|
+-------------------+----------+
|              31.89|      2015|
|              32.61|      2013|
|              32.28|      2014|
|              33.48|      2012|
|              32.91|      2016|
|              30.64|      2010|
|              32.28|      2011|
|              32.03|      2017|
+-------------------+----------+

--- 0.4237940311431885 seconds ---


In [114]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table df")

DataFrame[]

In [115]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('df')

False